In [1]:
%pip install flask scikit-learn pandas python-docx


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 244.3/244.3 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import docx
import pandas as pd

In [15]:
import pandas as pd
from docx import Document

# Load the Word documents
resume_doc = Document("Resume.docx")
jd_doc = Document("Job_description.docx")


resume_text = [para.text for para in resume_doc.paragraphs if para.text.strip() != '']
jd_text = [para.text for para in jd_doc.paragraphs if para.text.strip() != '']

# Repeat the 'international' and 'local' values for each row
international = ['Yes']
local = ['Yes'] 

df1=pd.DataFrame(resume_text,columns=['words'])
df2=pd.DataFrame(jd_text,columns=['words'])

In [16]:
df1.head()

,words
0,Pranav Belmannu
1,North Carolina <> 518-977-8939 <> belmanuprana...
2,SUMMARY
3,Results-oriented master’s graduate in Business...
4,SKILLS


In [17]:
df2.head()

,words
0,About the job
1,Position:
2,Operations Data Scientist
3,Job Description:
4,Summary


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import pickle

# Sample data 
data = {
    'resume': [
        'Experienced software engineer with skills in Python and machine learning.',
        'Entry-level marketing specialist with a background in social media.',
        # Add more resumes
    ],
    'job_description': [
        'Looking for a software engineer with Python and machine learning experience.',
        'Seeking a marketing specialist with social media experience.',
        # Add more job descriptions
    ],
    'selected': [1, 0]  # 1 for selected, 0 for not selected
}

df = pd.DataFrame(data)

# Combine resume and job description
df['text'] = df['resume'] + " " + df['job_description']

# Split data
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['selected'], test_size=0.2, random_state=42)

# Create a pipeline
model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

# Train the model
model.fit(X_train, y_train)

# Save the model
with open('resume_model.pkl', 'wb') as f:
    pickle.dump(model, f)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

In [ ]:
from flask import Flask, request, render_template
import pickle
from docx import Document
import re

app = Flask(__name__)

# Load the pre-trained model
with open('resume_model.pkl', 'rb') as f:
    model = pickle.load(f)

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    text = '\n'.join([para.text for para in doc.paragraphs])
    return text

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        first_name = request.form.get('first_name')
        last_name = request.form.get('last_name')
        resume_file = request.files.get('resume')
        international = request.form.get('international')
        local = request.form.get('local')
        job_description = request.form.get('job_description')

        # Extract text from resume file
        resume_text = extract_text_from_docx(resume_file)

        # Prepare input for the model
        input_text = f'{resume_text} {job_description}'

        # Predict
        prediction = model.predict([input_text])[0]

        if prediction == 1:
            result = 'The resume can be selected.'
        else:
            result = 'The resume cannot be selected.'

        return render_template('result.html', result=result)

    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)
